In [1]:
import numpy as np
import itertools
import sqlite3
import os
import re

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import trange

In [2]:
conn_src = sqlite3.connect("../Precedents.db")
conn_dst = sqlite3.connect("../Precedents_Keyword.db")
curs_src = conn_src.cursor()
curs_dst = conn_dst.cursor()

sql = "SELECT * FROM Precedents_IC"
curs_src.execute(sql)

fetchedData = curs_src.fetchall()
length = len(fetchedData)
fetchedData = [list(fetchedData[x]) for x in range(length)]

In [3]:
for x in trange(length):
    output_file = open('keyword_result.txt', 'a', encoding = 'utf-8')
    filepath = "C:/Temp/Cases/판례내용/{}.txt".format(fetchedData[x][0])
    
    keyword = ""

    if os.path.getsize(filepath) != 0:
        with open(filepath) as file:
            doc = file.read()
    
        okt = Okt()
        tokenized_doc = okt.pos(doc)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

        n_gram_range = (2, 3)

        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()

#        print('trigram 개수 :',len(candidates))
#        print('trigram 다섯개만 출력 :',candidates[:5])

        model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
        doc_embedding = model.encode([doc])
        candidate_embeddings = model.encode(candidates)

        top_n = 30
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        rslt = [candidates[index] for index in distances.argsort()[0][-top_n:]]
        for element in rslt:
            output_file.write(element + ' ')
    else:
        keyword = "내용없음"
        output_file.write(keyword)
    
    output_file.write('\n')
    output_file.close()
    curs_dst.execute(f"INSERT INTO Precedents_Keyword Values (\"{fetchedData[x][0]}\", \"{keyword}\")")

100%|██████████| 317/317 [5:20:22<00:00, 60.64s/it]    


이 밑부터는 초기(221103) 테스트코드

In [15]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('【', 'Foreign'), ('피고인', 'Noun'), ('】', 'Foreign'), ('【', 'Foreign'), ('검사', 'Noun'), ('】', 'Foreign'), ('최세훈', 'Noun'), ('【', 'Foreign'), ('변호인', 'Noun')]
명사 추출 : 피고인 검사 최세훈 변호인 변호사 이경환 주문 피고인 징역 처 이 판결 선고 전 구금일수 위 형 산입 공소 사실 중 부동산 매물 정보 저장 파일 침해 부분 무죄 이유 범죄사실 피고인 지방법원 정보통신망 이용 촉진 및 정보보호 등 관 법률 위반 죄 징역 집행유예 선고 현재 그 유예기간 중 데 한게임 아이디 패스워드 해당 아이디 보유 사이버 포커 를 현금 목적 인터넷 사이트 회원 가입 시 대부분 사람 아이디 패스워드 사용 점 착안 보안 취약 웹사이트 해킹 회원 아이디 비밀번호 소외 공모 경 경 사이 서울 성동구 송정동 프로게이머 피시방 부동산개발 관련 정보 업체 코리아 셋 인베스트먼트 주 의 인터넷 사이트 접속 위 회사 사용 프로그램 의 실행 취약점 이용 위해 위 사이트 문서 소스코드 조작 아이디 의 문자 정보 숫자 뒤 웹서버 입력 를 실행 수 명령어 아이디 란 입력 함 위 회사 접속 접근권한 정보통신망 침입 나 경 위 프로게이머 피시방 부동산 매물 중개회사 셋 메이커 주 의 인터넷 사이트 접속 위 항 방법 위 회사 접속 접근권한 정보통신망 침입 그 무렵 장소 항 위 회사 접속 후 권한 실행 수 상태 이나 의 명령어 실행 명의 회원정보 저장 파일 다운로드 사이트 내 디렉토리 보관 정보통신망 보관 타인 비밀 침해 경 위 프로게이머 피시방 인터넷 결혼 정보 회사 주 듀오 인터넷 사이트 접속 위 항 방법 위 회사 접속 접근권한 정보통신망 침입 그 무렵 장소 항 위 회사 접속 후 권한 실행 수 상태 이나 의 명령어 실행 약 회원정보 저장 파일 다운로드 위 나 항의 디렉토리 보관 정보통신망 보관 타인 비밀 침해 경 서울

In [16]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 656
trigram 다섯개만 출력 : ['가입 대부분' '가입 대부분 사람' '가중 형법' '가중 형법 전단' '개발 사용']


In [17]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [25]:
top_n = 30
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['구금일수 산입 공소', '항의 디렉토리 보관', '선고 구금일수 산입', '서울 성동구', '포함 피고인 다운로드', '미결구금일수 산입 형법', '악성코드 게시판 업로드', '대한 경찰 피의자신문조서', '비밀 침해 서울', '박정인 대한 경찰', '서울 서초구', '송정동 프로게이머 피시방', '피고인 다운로드 저장', '업체 코리아', '정보 업체 코리아', '성동구 송정동', '이용 일명 악성코드', '사이 서울 성동구', '침해 서울 서초구', '성동구 송정동 프로게이머', '코리아 인베스트먼트', '서울 성동구 송정동', '업체 코리아 인베스트먼트', '코리아 인베스트먼트 인터넷', '전단 미결구금일수 산입', '서울 서초구 서초동', '대한 검찰 피의자신문조서', '경찰 피의자신문조서 박정인', '피의자신문조서 박정인', '피의자신문조서 박정인 대한']
